In [1]:
import pandas as pd
import numpy as np
import sklearn
import os
import datetime
import time
import git
import sys
import math
from geopy.distance import geodesic
from sklearn.cluster import _kmeans

In [2]:
repo = git.Repo("./", search_parent_directories=True)
homedir = repo.working_dir
datadir = f"{homedir}" + "/models/processing/USA/County_Based/"

In [3]:
#helper functions
def logFunc(x):
    if x < 0.01:
        x = 0.01
    return math.log10(x)

# Convert longitude latitude pair to x, y, z Cartesian coordinates\n
def convertPts(pair):
    lon = pair[0]
    lat = pair[1]
    R = 3958.8
    lonRad = lon * math.pi / 180
    latRad = lat * math.pi / 180
    x = R * math.cos(latRad) * math.cos(lonRad)
    y = R * math.cos(latRad) * math.cos(lonRad)
    z = R * math.sin(lat)
    return (x, y, z)

def getX(x):
    return x[0]

def getY(x):
    return x[1]

def getZ(x):
    return x[2]

In [4]:
#Neighbor Data
neighborcounties = pd.read_csv(f"{homedir}/models/processing/USA/County_Based/neighborcounties.csv", index_col = 0)

# read in the files, load as dataframe
Age_Race = pd.read_csv(datadir + 'Age_Race_Filled.csv')
Population = pd.read_csv(datadir + 'Total_Pop')
Density = pd.read_csv(datadir + 'Density.csv')
JHU = pd.read_csv(datadir + 'aggregate_jhu_filled.csv')
Berkeley = pd.read_csv(datadir + 'Aggregate_Berkeley.csv')
Policies = pd.read_csv(datadir + 'Policy_Transit.csv')
Geography = pd.read_csv(datadir + 'County_Centers.csv')
Health = pd.read_csv(datadir + 'County_Health.csv')
Transit = pd.read_csv(datadir + 'Transit.csv')

Data = pd.DataFrame()

DataBasic = pd.DataFrame()
DataDemographics = pd.DataFrame()
DataHealth = pd.DataFrame()
DataGeography = pd.DataFrame()

In [5]:
Data['FIPS'] = Geography['fips']

# fix population
Population.columns = ['fips', 'Population']

# drop US territories, train separate models for them
FipsSet = []
counter = 0
for row in Data.iterrows():
    row = row[1][0]
    if math.floor(row / 1000) > 56:
        Data = Data.drop([counter], axis = 0)
    else:
        FipsSet.append(float(row))
    counter += 1
    
# edit county centers
counter1 = 0
for row in Geography.iterrows():
    if row[1][1] not in FipsSet:
        Geography = Geography.drop([counter1], axis = 0)
    counter1 += 1

In [14]:
# Nature of the county, includes policies

Data['Pop'] = Population['Population']
Data['Pop'] = Data['Pop'].div(2000000.0) # population max is 5

Data['Density'] = Density['2010 Density per square mile of land area - Population']
Data['Density'] = Data['Density'].div(5000.0) # density max is around 14

Data['Area'] = Density['Area in square miles - Total area']
Data['Area'] = Data['Area'].div(40000.0) #max around 4

Data['UrbanRural'] = JHU['Rural-urban_Continuum Code_2013']
Data['UrbanRural'] = Data['UrbanRural'].div(1.5) # urban rural max is 7

Data['EconType'] = JHU['Economic_typology_2015']
Data['EconType'] = Data['EconType'].div(1.5) # economic typology max is 3

# Policies
Data['Policies'] = Policies['Score']
Data['Policies'] = Data['Policies'].div(2) # policies max is just above 4.5

# Typical immigration in/out. Proxy for being a sink/source in flows
Data['Movement'] = JHU['R_NET_MIG_2018']
Data['Movement'] = Data['Movement'].div(20.0) #range around -3.5 to 3.5

Data['Transit'] = Transit['Score']

In [7]:
# Demographics of the county

# Age distribution
Data['65+'] = Age_Race['65 to 74 years'] + Age_Race['75 to 84 years'] + Age_Race['85 years and over']
Data['65+'] = Data['65+'] / Population['Population']
Data['65+'] = Data['65+'].mul(2400) # 65+ max is 4.5

# Race/gender
Data['Male'] = Berkeley['FracMale2017']
Data['Male'] = Data['Male'].mul(6) # male max is 2, generally 1

Data['AfricanAmer'] = Age_Race['Exclusively Black or African American'] + Age_Race['Hispanic or Latino (of any race)!!Puerto Rican']
Data['AfricanAmer'] = Data['AfricanAmer'] / Population['Population']
Data['AfricanAmer'] = Data['AfricanAmer'].mul(4500) # African American max is 6
     
# Politics/education/income/economy
Data['CollegePlus'] = JHU['Percent of adults completing some college or associate\'s degree 2014-18']
Data['CollegePlus'] = Data['CollegePlus'].div(10) # education max is 3, generally around 1-1.5

Data['Income'] = JHU['Median_Household_Income_2018']
Data['Income'] = Data['Income'].div(40000) # income max is 4, generally around 2  

Data['Unemployed'] = JHU['Unemployment_rate_2018']
Data['Unemployed'] = Data['Unemployed'].div(3) # unemployed max around 3
                         
    # need puerto rico voting patterns
Data['Dems'] = Berkeley['FracDem']
Data['Dems'] = Data['Dems'].mul(7.5) # Dems max is 5, generally around 1.5

In [8]:
#Health care of the county

Data['Hospitals'] = Berkeley['#Hospitals'] * 10000.0 / Population['Population']
Data['Hospitals'] = Data['Hospitals'].div(30.0)

Data['HospBeds'] = Health['licensed_beds'] / Population['Population'] # around 2-3

Data['ICUBeds'] = Berkeley['#ICU_beds']
Data['ICUBeds'] = Data['ICUBeds'] / Population['Population']
Data['ICUBeds'] = Data['ICUBeds'].mul(10) # around 11. Outliers ok, very important statistic

#note: not considering comorbidities
Data['HeartDiseaseMort'] = Berkeley['HeartDiseaseMortality'] 
Data['HeartDiseaseMort'] = Data['HeartDiseaseMort'].div(60) # max is 10, typically around 3-4

Data['StrokeMort'] = Berkeley['StrokeMortality']
Data['StrokeMort'] = Data['StrokeMort'].div(10) # max is 7-8, generally around 5

Data['Diabetes'] = Berkeley['DiabetesPercentage'] 
Data['Diabetes'] = Data['Diabetes'].mul(9) # max 3

Data['Smokers'] = Berkeley['SmokersPercentage'] 
Data['Smokers'] = Data['Smokers'].mul(9) # max 3

In [9]:
#Geography

Data['pLonLat'] = list(zip(Geography.pclon10, Geography.pclat10)) # population weighted
Data['pLonLat'] = Data['pLonLat'].values

Data['XYZ'] = Data['pLonLat'].apply(convertPts)

Data['xVal'] = Data['XYZ'].apply(getX)
Data['xVal'] = Data['xVal'].div(100)

Data['yVal'] = Data['XYZ'].apply(getY)
Data['yVal'] = Data['yVal'].div(100)

Data['zVal'] = Data['XYZ'].apply(getZ)
Data['zVal'] = Data['zVal'].div(100)

Data = Data.drop(columns=['pLonLat', 'XYZ'])

In [15]:
for column in Data.columns:
    print((column, max(Data[column]), Data[column].isnull().sum()))
print(len(Data.columns))

('FIPS', 56045, 0)
('Pop', 5.052861, 0)
('Density', 13.893679999999998, 0)
('Area', 3.6951157500000003, 0)
('UrbanRural', 6.0, 53)
('EconType', 3.3333333333333335, 53)
('Policies', 4.95, 9)
('Movement', 3.47, 53)
('Transit', 9.9, 4)
('65+', 4.007154643627509, 0)
('Male', 4.405507745266783, 29)
('AfricanAmer', 5.645851501883725, 0)
('CollegePlus', 5.7299999999999995, 53)
('Income', 3.50955, 53)
('Unemployed', 6.033333333333334, 53)
('Dems', 7.177139125639056, 29)
('Hospitals', 3.875968992248062, 29)
('HospBeds', 5.071428571428571, 660)
('ICUBeds', 4.945054945054945, 29)
('HeartDiseaseMort', 10.05, 29)
('StrokeMort', 9.99, 30)
('Diabetes', 2.97, 29)
('Smokers', 3.734217805860001, 29)
('xVal', 10.753779705773864, 0)
('yVal', 10.753779705773864, 0)
('zVal', 39.587931929159076, 0)
26


In [11]:
Data.head

<bound method NDFrame.head of        FIPS       Pop  Density      Area  UrbanRural  EconType  Policies  \
0      1001  0.027800  0.01836  0.015110    1.333333  0.000000      0.00   
1      1003  0.109011  0.02292  0.050683    2.000000  3.333333      0.15   
2      1005  0.012441  0.00620  0.022613    4.000000  2.000000      0.00   
3      1007  0.011200  0.00736  0.015654    0.666667  0.000000      0.05   
4      1009  0.028920  0.01778  0.016266    0.666667  0.000000      0.00   
...     ...       ...      ...       ...         ...       ...       ...   
3138  56037       NaN  0.00084  0.262277         NaN       NaN       NaN   
3139  56039       NaN  0.00106  0.105408         NaN       NaN       NaN   
3140  56041       NaN  0.00202  0.052189         NaN       NaN       NaN   
3141  56043       NaN  0.00076  0.056067         NaN       NaN       NaN   
3142  56045       NaN  0.00060  0.060000         NaN       NaN       NaN   

      Movement  Transit       65+  ...  Hospitals  HospBe

In [12]:
# functions to fill in Data's Nans

def fillcol(fips, value,neighborcounties, min_neighbors=2):
    #Takes in a column of fips codes, and any type of datafield with some NaNs,
    #Computes distance-weighted average of the value across all neighbors of NaN counties
    tic1 = time.time()
    #Loading in the fips and value into proper dataframes
    #This is the df with only nan values
    df = pd.DataFrame(data = [fips,value])
    df.columns = ['FIPS', 'Values']
    df.Values = df.Values.astype(float)
    df = df.set_index('FIPS')
    
    #creating new column to set to the current dataframe values
    newcol = []
    for ind in df.index:
        #for any entries with NaNs
        if np.isnan(df['Values'][ind]):
            #list of neighbors for NaN county
            neighbors = list(neighborcounties[neighborcounties['orgfips'] == ind]['adjfips'])
            nonzero = 0
            weightedval = 0
            totalinvdist = 0
            totaldist = 0
            vals = 0
            #iterates though neighbors of NaN county with non-NaN entires
            for n in neighbors:
                if n in df.index:
                    if ~np.isnan(df['Values'][n]):
                        #Getting weighted values, using 1/dist as a scalar to show closer distance counts more
                        nonzero += 1
                        dist = list(neighborcounties.query('orgfips == ' + str(ind) + ' and adjfips == ' + str(n))['Pop_10'])[0]
                        totalinvdist += (1/dist)**1
                        weightedval += ((1/dist)**1)*df['Values'][n]
            #If there are at least 2 neighbors (this can be adjusted)
            if nonzero >= min_neighbors:
                newcol.append(weightedval/(totalinvdist))
            else:
                newcol.append(np.nan)
        else:
            newcol.append(df['Values'][ind])
    toc1 = time.time()
    print(toc1 - tic1)
    return newcol

def fillfixed(colname, data, code, neighborcounties):
    #Method to fill up the google mobility data
    #Uses colname to designate which column to fill
    numnans = len(data[np.isnan(data[colname])])
    while numnans > 0:
        print(numnans)
        tempnum = numnans
        #Creating the filled column from method
        newcol = fillcol(data[code], data[colname], neighborcounties)
        data[colname] = newcol
        numnans = len(data[np.isnan(data[colname])])
        #Checking if the number of nans changes
        if tempnum == numnans:
            #if number doesnt change, try again with only 1 neighbor, otherwise quit
            newcol = fillcol(data[code], data[colname], neighborcounties)
            data[colname] = newcol
            numnans = len(data[np.isnan(data[colname])])
            if tempnum == numnans:
                numnans = 0     
    return data

In [13]:
# Filling in columns of dataframe by nearest neighbor analysis

cols = list(Data.columns)[1:]
for col in cols:
    Data = fillfixed(col, Data, 'FIPS', neighborcounties)
Data = Data.dropna()

29


ValueError: Length mismatch: Expected axis has 3143 elements, new values have 2 elements

In [ ]:
for column in Data.columns:
    print((column, Data[column].isnull().sum()))
print(len(Data.columns))

In [ ]:
# getting specific dataframes for what we want to cluster

DataBasic = Data[['Pop', 'Density', 'Area', 'UrbanRural', 'EconType', 'Policies', 'Movement']]

DataDemographics = Data[['Male', 'AfricanAmer', 'CollegePlus', 'Income', 'Unemployed', 'Dems']]

DataHealth = Data[['HospBeds', 'ICUBeds', 'HeartDiseaseMort', 'StrokeMort', 'Diabetes', 'Smokers']]

# approximate distances between counties
# we double count Urban Rural, population, density, size since don't cluster on the entire Data dataframe
# so that nearby urban counties are closer than a rural county adjacent from an urban county
DataGeography = Data[['Pop', 'Density', 'Area', 'UrbanRural', 'xVal', 'yVal', 'zVal']]

In [ ]:
# plot k vs error on a graph, decide optimal k via elbow method

In [ ]:
# plot the clusters across the country to visualize